# Soluzione 'The Maze Challenge' - Davide Pruscini (prushh)

## <font color='#428bca'>Engine</font>
Al fine di risolvere le varie Quest è stato necessario capire il funzionamento dell'engine, messo a disposizione grazie ai tre file eseguibili (rispettivamente per Linux, MacOS o Windows) e come interagire con esso tramite il modulo **mazeClient**.
Al suo interno troviamo due classi (**Commands**, **\_MazeCommand**) e la funzione `send_command(command)` che ci permetterà di:
* Ricavare informazioni su nodo corrente, *GET_STATE*
    * {
          "userX": 14,
          "userY": 10,
          "userVal": 32,
          "Neighbors": [
            {
              "x": 14,
              "y": 9,
              "val": 82
            },
            {
              "x": 14,
              "y": 11,
              "val": 71
            }
          ]
        }
* Muovere il cursore tra i vari nodi
    * (x, y+1), *MOVE_LEFT*
    * (x, y-1), *MOVE_RIGHT*
    * (x+1, y), *MOVE_UP*
    * (x-1, y), *MOVE_DOWN*
* Terminare l'esecuzione dell'engine
    * *EXIT*

Selezionando la finestra del processo in esecuzione, è inoltre possibile usare le frecce direzionali per muoversi in maniera molto rapida tra le celle.
Modificando il file **seed.maze** si potranno ottenere ulteriori labirinti distinti.

## <font color='#428bca'>Argparse</font>
```
usage: main.py file_path [--no_stats] [-c] [-d] [-h]

                        'The Maze Challenge' by MircoT
                                Solution by prushh
        ------------------------------------------------------------
        There are two modes of use:
         - Classic maze solver with stats
         - Controller mode (disable stats)
        You must specify mazeEngine.ext path for parallel execution.


positional arguments:
  file_path         executable file to start engine

optional arguments:
  -h, --help        show this help message and exit
  -c, --controller  controller mode for interact with engine
  --no_stats        hide statistical information
  -d, --debug       print debug information and more

```

## <font color='#428bca'>Quests</font>
### 1. Muoviti e raccogli statistiche sul labirinto
Questa è stata la Quest a mio parere più impegnativa, non tanto per la raccolta di alcuni dati statistici ma per la realizzazione di un algoritmo che funzionasse correttamente per tutti i seed.<br>
Avendo a disposizione per ogni nodo una lista contenente i vicini per tutte e 8 le direzioni (4 raggiungibili + 4 diagonali), ho pensato di adattare il problema della visita in profondità di un grafo (DFS), immaginando ogni cella come un vertice appartenente ad esso.<br>
Una prima implementazione iterativa comprendeva l'ausilio di una *Stack* ma ho avuto problemi nel tornare indietro ad ispezionare nuovamente i *Neighbors* e di conseguenza ad aggiornare l'interfaccia; così ho optato per una soluzione ricorsiva:

In [ ]:
def explore(self, cur_node: dict, last_act: str):
        '''
        Core for maze solver. It is a recursive function
        inspired to Depth-First-Search logic.
        '''
        neighbors = cur_node['Neighbors']
        for neighbor in neighbors:
            if neighbor in self.__get_valid_neighbors(cur_node):
                if neighbor not in self.visited:
                    self.visited.append(neighbor)
                    self.__increment_coord_color(neighbor)
                    self.__increment_color(neighbor['val'])

                    action = self.__get_action(cur_node, neighbor)
                    new_node = to_dict(get_response(action))
                    if args.debug:
                        print(action)
                        sleep(0.5)

                    self.explore(new_node, action)
            elif neighbor not in self.visited:
                self.known.append(neighbor)
        # There are not other valid near node,
        # reverse last action and go back
        reverse_act = self.__reverse_action(last_act)
        get_response(reverse_act)
        if args.debug:
            print(reverse_act)
            sleep(0.5)

 - Per ogni vicino in *Neighbors* controllo se è raggiungibile, escludo quindi i nodi diagonali che andrò a considerare nell'altro ramo di selezione
 - Più internamente, se questo vicino non è stato già visitato:
     - lo aggiungo alla lista dei visitati
     - aggiorno le varie strutture per memorizzare dati statistici
     - ricavo l'azione per raggiungerlo e sposto il cursore
     - richiamo la funzione `explore()` con i nuovi parametri
 - La chiamata ricorsiva è terminata, inverto l'ultima azione e torno indietro
 
Infine, per avere tutti i nodi memorizzati in una sola struttura, è necessario richiamare la seguente funzione che aggiunge alla lista dei nodi visitati i nodi diagonali conosciuti durante il tragitto, ma non raggiungibili fisicamente.

In [ ]:
def __fix_known(self):
    '''
    Add the known neighbors to the visited ones.
    '''
    for neighbor in self.known:
        if neighbor not in self.visited:
            print(neighbor)
            self.visited.append(neighbor)
            self.__increment_coord_color(neighbor)
            self.__increment_color(neighbor['val'])

### 2. Conta le celle totali nel labirinto (Bianche, Rosse, Blu, Verdi)
### 3. Conta il numero di celle Rosse, Blu e Verdi
Come detto in precedenza la raccolta dei dati non è stata complicata, ho utilizzato un dizionario inizializzato nel modo seguente:
```python
self.colors_count = {
            'white': 0,
            'blue': 0,
            '#00ff00': 0,
            'red': 0
        }
```
Con l'ausilio di questa struttura posso andare direttamente ad incrementare il valore specificando la *key*, abbiamo visto però che il colore rappresentato da *userVal* viene fornito sotto forma di codice, bisogna quindi effettuare una conversione:

In [ ]:
def __get_color_name(self, color: int) -> str:
    '''
    Returns color name specified by int code.
    ex: 32 -> white
    '''
    colors_code = {
        32: 'white',
        66: 'blue',
        71: '#00ff00',
        82: 'red'
    }
    return colors_code[color]

Una volta ottenuta la stringa che rappresenta il colore si può provvedere ad aggiornare le frequenze:

In [ ]:
def __increment_color(self, color: int):
    '''
    Update the frequency of colors find in the maze.
    '''
    self.colors_count[self.__get_color_name(color)] += 1

Da `self.colors_count` possiamo quindi facilmente ricavare:
 * numero totale di celle, utilizzando `sum(self.colors_count.values())`
 * numero di celle per ogni colore

Per riprendere confidenza con i plot questi dati sono stati visualizzati graficamente, utilizzando la funzione `plt_colors_dist(dict_)` reperibile in **statistics.py**

### 4. Traccia l'istogramma della distribuzione della frequenza di colore per le coordinate X e Y<a name='quest_four' />
Anche in questo caso è stato utilizzato un dizionario, con all'interno due *key* già presenti:
```python
self.colors_xy = {'x': {}, 'y': {}}
```
L'idea è quella di aggiungere per ogni nodo ispezionato la propria coordinata x come *key* in `self.colors_xy['x']` ed equivalentemente per y in `self.colors_xy['y']`. Dato che non conosciamo a priori le *sub-key* e bisognerà contare per ognuna di esse la frequenza con la quale si ripete un colore, si è pensato di usare la funzione `setdefault(keyname, value)` per l'inizializzazione di ogni coordinata nel seguente modo:

In [ ]:
def __increment_coord_color(self, node: dict):
    '''
    Update the color frequency distribution
    for XY coordinates.
    '''
    x = node['x']
    y = node['y']
    c_code = node['val']
    dict_ = {
            'white': 0,
            'blue': 0,
            '#00ff00': 0,
            'red': 0
    }

    # If key does not exists create
    # and fill it with default value
    self.colors_xy['x'].setdefault(x, dict_)
    self.colors_xy['y'].setdefault(y, dict_)
    self.colors_xy['x'][x][self.__get_color_name(c_code)] += 1
    self.colors_xy['y'][y][self.__get_color_name(c_code)] += 1

Come espressamente richiesto questi dati sono stati visualizzati graficamente, utilizzando la funzione `plt_xy_dist(xy_freq, old_xy_freq)` reperibile in **statistics.py**

In [ ]:
def plt_xy_dist(xy_freq: dict, old_xy_freq: dict):
    '''
    Plot the histogram of the color frequency
    distribution for X e Y coordinates.
    If old_freq is not empty, show the previous distribution.
    '''
    # Prepares colors list and labels
    colors = xy_freq['x'][list(xy_freq['x'].keys())[0]].keys()
    title = 'Colors distribution for XY coordinates'
    title_x = 'X distribution'
    title_y = 'Y distribution'
    old_title_x = f'Old {title_x}'
    old_title_y = f'Old {title_y}'
    ylabel = 'Number of cells'

    # Creates pandas.DataFrame (more simple to manage) and sort them for index
    # Current maze information
    df_x = pd.DataFrame.from_dict(xy_freq['x'], orient='index').sort_index()
    df_y = pd.DataFrame.from_dict(xy_freq['y'], orient='index').sort_index()
    
    # Create subplot ad customize it
    fig, axes = _dark_subplots(nrows=2, ncols=2)
    fig.suptitle(title, fontsize=15)
    df_x.plot.bar(figsize=(12, 8), ax=axes[0][0], yticks=_get_max_tick(df_x), color=colors, legend=False)
    df_y.plot.bar(ax=axes[0][1], yticks=_get_max_tick(df_y), color=colors, legend=False)
    axes[0][0].set(title=title_x, ylabel=ylabel)
    axes[0][1].set(title=title_y)

    if len(old_xy_freq):
        # Old maze information
        old_df_x = pd.DataFrame.from_dict(old_xy_freq['x'], orient='index').sort_index()
        old_df_y = pd.DataFrame.from_dict(old_xy_freq['y'], orient='index').sort_index()

        # Create subplot ad customize it
        old_df_x.plot.bar(ax=axes[1][0], yticks=_get_max_tick(old_df_x), color=colors, legend=False)
        old_df_y.plot.bar(ax=axes[1][1], yticks=_get_max_tick(old_df_y), color=colors, legend=False)

        axes[1][0].set(title=old_title_x, ylabel=ylabel)
        axes[1][1].set(title=old_title_y)

    plt.show()

Nella <a href=#quest_seven>Quest7</a> verrà commentata la possibilità di confrontare le vecchie distribuzioni della frequenza di colore per X ed Y.

### 5. Traccia il labirinto esplorato come una mappa
Avendo a disposizione le coordinate *x* ed *y* per ogni nodo viene spontaneo pensare alla realizzazione di una matrice.<br>
Si è deciso di utilizzare la funzione `numpy.zeros(shape, dtype=float, order='C')` per creare un array 2D inizializzato a 0: le dimensioni di quest'ultimo saranno definite da una semplice operazione del tipo `max_coord - min_coord + 1` e verrà specificato il parametro `dtype`.<br>
Per ricavare i valori massimi e minimi che ci occorrono, la lista dei nodi è stata convertita in **pandas.DataFrame** così da effettuare questa operazione in maniera molto semplice. Si valorizzano poi le corrispettive celle della matrice con i valori *1, 2, 3, 4* a seconda del colore di ogni nodo.<br>
È stata infine scelta la funzione `matshow(A, fignum=None, \**kwargs)` per visualizzare la matrice creata, la quale prende in input anche cmap, oggetto **Colormap** generato da una lista di colori.

In [ ]:
def plt_map(visited: list):
    '''
    Plot the explored maze as a map.
    '''
    title = 'Map of the explored maze'
    cmap = ListedColormap(['black', 'white', 'blue', '#00ff00', 'red'])
    colors_code = {
        32: 1,
        66: 2,
        71: 3,
        82: 4
    }

    # Simple solution for max/min with pd.DataFrame
    df = pd.DataFrame(visited)
    min_x = df.x.min()
    min_y = df.y.min()
    max_x = df.x.max()
    max_y = df.y.max()

    # Returns a new 2-D array of int, filled with zeros
    matrix = np.zeros(_get_dimensions(max_x, min_x, max_y, min_y), dtype=int)
    for node in visited:
        x = max_x - node["x"]
        y = max_y - node["y"]
        # Fills matrix position with
        # the color_code converted
        matrix[x, y] = colors_code[node["val"]]

    # Create subplot ad customize it
    fig, ax = _dark_subplots()
    fig.suptitle(title, fontsize=15)
    ax.matshow(matrix, cmap=cmap)
    plt.xticks(_get_range(max_y, min_y), _get_range(max_y, min_y, True))
    plt.yticks(_get_range(max_x, min_x), _get_range(max_x, min_x, True))

    plt.show()

# <font color='#428bca'>Advanced Quests</font>
### 6. Prova a esplorare un labirinto con un seed diverso
Semplicemente per risolvere questa Quest è stato creato un file contenente una nuova 'chiave':<br>
`1594709668559390700`<br>
In fase di test è stato sostituito il contenuto del file **seed.maze**, per la memorizzazione è stato successivamente rinominato in **seed1.maze**

### 7. Confronta la distribuzione dell'istogramma con il labirinto precedente <a name='quest_seven' />
Come visto in precedenza nella <a href=#quest_four>Quest4</a>, la funzione `plt_xy_dist(xy_freq, old_xy_freq)` ci permette di tracciare graficamente la distribuzione della frequenza di colore per X ed Y.<br>
Ogni volta che finisce l'esplorazione di un labirinto viene richiamata la seguente funzione, disponibile in **functions.py**, che svolge le seguenti operazioni:
 * carica il file `previous_maze.pickle` se presente
 * confronta la vecchia distribuzione con la nuova
     * se sono uguali non viene aggiornato il file
     * se non sono uguali procedo scaricando la nuova distribuzione su file

In [ ]:
def get_pickle_obj(colors_xy: dict) -> dict:
    '''
    Check the old pickle file and compare it
    to the current color distribution.
    '''
    old_data = {}
    name_file = 'previous_maze.pickle'

    try:
        with gzip.open(name_file, 'rb') as handle:
            old_data = pickle.load(handle)
    except FileNotFoundError:
        pass
    if old_data == colors_xy:
        old_data = {}
        print("Color frequency distribution not saved, same as the previous one...")
    else:
        print("Save XY color frequency distribution to pickle file...")
        with gzip.open(name_file, 'wb') as handle:
            pickle.dump(colors_xy, handle, protocol=pickle.HIGHEST_PROTOCOL)

    return old_data

Se la lunghezza del dizionaro *old_xy_freq* in `plt_xy_dist()` è maggiore di 0, si procede con la preparazione dei vecchi dati da confrontare, altrimenti verrà visualizzata la distribuzione attuale con in basso due riquadri vuoti, ad indicare che non sono presenti cambiamenti.

### 8. Crea un client che consente lo spostamento nel labirinto con le frecce direzionali
Dato che l'*Engine* supporta nativamente l'utilizzo delle frecce direzionali si è deciso di organizzare i tasti nel seguente modo:
 * **WASD** per il movimento
 * **F** per l'ispezione
 * **E** per uscire

In una prima implementazione, per catturare i tasti premuti, era stato utilizzata la funzione `getch()` del pacchetto **msvcrt**, vista la compatibilità con i soli sistemi *Windows* si è deciso di utilizzare l'oggetto **Listener** dal pacchetto **pynput.keyboard** che dovrebbe essere indipendente dal *SO* (non ho avuto modo di provare).

In [ ]:
class Controller:
    '''
    Class that contains methods to use client controller.
    '''
    def __init__(self, debug):
        '''
        Initialize flag for prints debug information,
        creates act_map for key-action association.
        '''
        self.debug = debug
        self.act_map = {
            'w': command.MOVE_UP,
            'a': command.MOVE_LEFT,
            's': command.MOVE_DOWN,
            'd': command.MOVE_RIGHT,
            'f': command.GET_STATE,
            'e': command.EXIT
        }

    def explore_maze(self):
        '''
        Listen for keys pressed.
        '''
        with Listener(on_press=self._on_press) as listener:
            listener.join()

    def _on_press(self, key):
        '''
        Sends the action to do based on key pressed.
        '''
        try:
            # Check if key correspond to an action
            if key.char in self.act_map.keys():
                action = self.act_map[key.char]
                if self.debug:
                    print(action)
                if key.char == 'e':
                    return False
                tmp = get_response(action)
                if key.char == 'f':
                    pprint(to_dict(tmp))
        except AttributeError:
            # Cleans terminal after an illegal key
            self._flush_input()

    def _flush_input(self):
        '''
        Flush the keyboard input buffer.
        '''
        try:
            # Windows
            import msvcrt
            while msvcrt.kbhit():
                msvcrt.getch()
        except ImportError:
            # Linux, MacOS
            import sys
            import termios
            termios.tcflush(sys.stdin, termios.TCIOFLUSH)
